In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

import pandas as pd

In [2]:
#loops thru dataset files, turns each image into an array, classifies and returns list of arrays
def img_array(path, class_num, size):
    data =[]
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (size,size))
        data.append(np.array([new_array, class_num]))
    return(data)

def prepare(filepath):
    IMG_SIZE = 120
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
#returns model prediction score for a single image

def get_prediction(filepath, model):
    tester = prepare(filepath)
    tester = tester.astype(np.float32)/255
    return(float(model.predict([tester])))

#iterates thru images in a folder, returns list of model prediction scores for each image
def mass_predict(path, model):
    data =[]
    for img in os.listdir(path):
        #print(os.path.join(path,img))
        prediction = get_prediction(os.path.join(path,img), model)
        data.append(prediction)
    return data

#returns dataframe of images with their corresponding file names and model prediction scores
def mass_predict_df(path,model):
    df = pd.DataFrame(columns = ['filename', 'score'])
    filenames = []
    scores = []
    for img in os.listdir(path):
        #print(os.path.join(path,img))
        prediction = get_prediction(os.path.join(path,img), model)
        filenames.append(img)
        scores.append(prediction)
    df['filename'] = pd.Series(filenames)
    df['score'] = pd.Series(scores)
    return df

#prepares image for input into model #watch filesize


# gets class based on threshold score
def verify(score, threshold):
    if score >= threshold:
        return('with_mask')
    else:
        return('without_mask')
    
# gets classed based on model prediction
def get_class(filepath, model):
    tester = prepare(filepath)
    tester = tester.astype(np.float32)/255
    return(float(model.predict_classes([tester])))

# returns df of images with corresponding filenames and class   
def mass_predict_df_class(path,model):
    df = pd.DataFrame(columns = ['filename', 'score'])
    filenames = []
    scores = []
    for img in os.listdir(path):
        #print(os.path.join(path,img))
        prediction = get_class(os.path.join(path,img), model)
        filenames.append(img)
        scores.append(prediction)
    df['filename'] = pd.Series(filenames)
    df['score'] = pd.Series(scores)
    return df
    
def predict_and_show(filepath, model, threshold):
    img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    plt.imshow(img, cmap="gray")
    plt.show()
    print(verify(get_prediction(filepath, model), threshold))
    print('score = ' + str(get_prediction(filepath, model)))
    
#evaluate df without mask
def eval_df_without_mask(df_without_mask):
    print('df without mask')
    print(df_without_mask.describe())
    stdev_over_mean1 =  float(df_without_mask.describe()[2:3]['score']) / float(df_without_mask.describe()[1:2]['score'])
    print('stdev over mean = ' + str(stdev_over_mean1))
    mean_plus_4_sigma1 = float(df_without_mask.describe()[1:2]['score']) + 4*float(df_without_mask.describe()[2:3]['score']) 
    mean_plus_3_sigma1 = float(df_without_mask.describe()[1:2]['score']) + 3*float(df_without_mask.describe()[2:3]['score'])
    print('mean plus 4 sigma = ' + str(mean_plus_4_sigma1))
    print('mean plus 3 sigma = ' + str(mean_plus_3_sigma1))

#evaluate df with mask
def eval_df_with_mask(df_with_mask):
    print('df with mask')
    print(df_with_mask.describe())
    stdev_over_mean2 =  float(df_with_mask.describe()[2:3]['score']) / float(df_with_mask.describe()[1:2]['score'])
    print('stdev over mean = ' + str(stdev_over_mean2))
    mean_minus_4_sigma1 = float(df_with_mask.describe()[1:2]['score']) - 4*float(df_with_mask.describe()[2:3]['score']) 
    mean_minus_3_sigma1 = float(df_with_mask.describe()[1:2]['score']) - 3*float(df_with_mask.describe()[2:3]['score'])
    print('mean minus 4 sigma = ' + str(mean_minus_4_sigma1))
    print('mean minus 3 sigma = ' + str(mean_minus_3_sigma1))

In [3]:
# pull dataset

path_without_mask = 'dataset\without_mask_3'
class_num_without_mask = 0

path_with_mask = 'dataset\with_mask_4'
class_num_with_mask = 1

size = 120

without_mask = img_array(path_without_mask, class_num_without_mask ,size)

with_mask = img_array(path_with_mask, class_num_with_mask, size)

data = without_mask + with_mask

X = []
y = []
scale_factor = 255
for features, label in data:
    X.append(features/scale_factor)
    y.append(label)
X = np.array(X).reshape(-1, size, size, 1)
y = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 5, stratify = y)

In [10]:
X_train.shape

(1730, 120, 120, 1)

In [9]:
#build and train NN

hyparam ={'loss':'binary_crossentropy',
         'opt':'Adam',
         'epochs': 10,
         'batch_size':16,
         'validation_data':(X_test, y_test) ,
         'lr':0.001}

model = Sequential()

#Conv
model.add(Conv2D(128, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

#Conv
model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

#Conv
model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

#FC
model.add(Flatten())
model.add(Dense(128))

#output layer
model.add(Dense(1)) 
model.add(Activation('sigmoid'))

model.compile(loss= hyparam['loss'], optimizer = hyparam['opt'], metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = hyparam['batch_size'], epochs = hyparam['epochs'], validation_data = hyparam['validation_data'])

Train on 1730 samples, validate on 577 samples
Epoch 1/10
1730/1730 [==============================] - 117s 67ms/sample - loss: 0.7499 - accuracy: 0.5659 - val_loss: 0.6701 - val_accuracy: 0.5927
Epoch 2/10
1730/1730 [==============================] - 116s 67ms/sample - loss: 0.6330 - accuracy: 0.6491 - val_loss: 0.6148 - val_accuracy: 0.6794
Epoch 3/10
1730/1730 [==============================] - 115s 66ms/sample - loss: 0.5426 - accuracy: 0.7324 - val_loss: 0.4506 - val_accuracy: 0.7972
Epoch 4/10
1730/1730 [==============================] - 115s 66ms/sample - loss: 0.4729 - accuracy: 0.7734 - val_loss: 0.4142 - val_accuracy: 0.8336
Epoch 5/10
1730/1730 [==============================] - 116s 67ms/sample - loss: 0.4355 - accuracy: 0.8081 - val_loss: 0.4059 - val_accuracy: 0.8094
Epoch 6/10
1730/1730 [==============================] - 120s 70ms/sample - loss: 0.4082 - accuracy: 0.8116 - val_loss: 0.3497 - val_accuracy: 0.8614
Epoch 7/10
1730/1730 [==============================] - 115

In [21]:
model.save('2020-06-14-1746-Conv2D.model')

W0614 17:53:19.970312 17368 deprecation.py:506] From c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [30]:
#build and train NN

hyparam ={'loss':'binary_crossentropy',
         'opt':'Adam',
         'epochs': 10,
         'batch_size':16,
         'validation_data':(X_test, y_test) ,
         'lr':0.001}

model = Sequential()

#Conv
model.add(Conv2D(128, (4,4), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.3))

#Conv
model.add(Conv2D(128, (4,4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

#Conv
model.add(Conv2D(128, (4,4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

#FC
model.add(Flatten())
model.add(Dense(64))

#output layer
model.add(Dense(1)) 
model.add(Activation('sigmoid'))

model.compile(loss= hyparam['loss'], optimizer = hyparam['opt'], metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = hyparam['batch_size'], epochs = hyparam['epochs'], validation_data = hyparam['validation_data'])

Train on 1730 samples, validate on 577 samples
Epoch 1/10
1730/1730 [==============================] - 181s 105ms/sample - loss: 0.7179 - accuracy: 0.5838 - val_loss: 0.6643 - val_accuracy: 0.6014
Epoch 2/10
1730/1730 [==============================] - 171s 99ms/sample - loss: 0.6140 - accuracy: 0.6682 - val_loss: 0.5806 - val_accuracy: 0.7002
Epoch 3/10
1730/1730 [==============================] - 205s 118ms/sample - loss: 0.5174 - accuracy: 0.7543 - val_loss: 0.4142 - val_accuracy: 0.8232
Epoch 4/10
1730/1730 [==============================] - 198s 115ms/sample - loss: 0.4023 - accuracy: 0.8145 - val_loss: 0.8893 - val_accuracy: 0.6690
Epoch 5/10
1730/1730 [==============================] - 198s 115ms/sample - loss: 0.3766 - accuracy: 0.8370 - val_loss: 0.3756 - val_accuracy: 0.8544
Epoch 6/10
1730/1730 [==============================] - 181s 105ms/sample - loss: 0.3108 - accuracy: 0.8763 - val_loss: 0.3462 - val_accuracy: 0.8648
Epoch 7/10
1730/1730 [==============================] 

In [31]:
model.save('2020-06-14-1901-Conv2D.model')

In [ ]:
N = hyparam['epochs']
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [6]:
# model evaluation on devset
#path1 is without mask, path2 is with mask

path1 = 'dev_set\without'
path2 = 'dev_set\with'
model = tf.keras.models.load_model('2020-06-14-1901-Conv2D.model')

df_without_mask = mass_predict_df(path1, model)
df_with_mask = mass_predict_df(path2, model)

eval_df_without_mask(df_without_mask)
print('\n')
eval_df_with_mask(df_with_mask)

print('\n')
x = (float(df_with_mask.describe()[1:2]['score'])-float(df_without_mask.describe()[1:2]['score'])) / (float(df_with_mask.describe()[1:2]['score'])+float(df_without_mask.describe()[1:2]['score']))
print('normalized mean difference = ' +  str(x))


df without mask
           score
count  23.000000
mean    0.210567
std     0.151885
min     0.013049
25%     0.115276
50%     0.182678
75%     0.260266
max     0.667010
stdev over mean = 0.7213139672011566
mean plus 4 sigma = 0.818107613782082
mean plus 3 sigma = 0.6662225209024244


df with mask
           score
count  25.000000
mean    0.484820
std     0.247167
min     0.102948
25%     0.315532
50%     0.468490
75%     0.552116
max     0.999589
stdev over mean = 0.5098121932638036
mean minus 4 sigma = -0.5038485039810165
mean minus 3 sigma = -0.25668139871506584


normalized mean difference = 0.39438846566734237


In [24]:
mass_predict_df_class(path2,model).describe()

,score
count,25.000000
mean,0.720000
std,0.458258
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
mass_predict_df(path2,model).sort_values(by= 'score',ascending =True)

,filename,score
24,with_test8.JPG,0.102948
17,with_test3.JPG,0.185200
6,with_test17.JPG,0.189225
18,with_test30.JPG,0.192101
11,with_test21.JPG,0.287027
13,with_test24.JPG,0.295773
23,with_test6.JPG,0.315532
15,with_test26.JPG,0.324634
12,with_test23.JPG,0.325715
4,with_test14.JPG,0.330399


In [8]:
#path1 is without

mass_predict_df(path1,model).sort_values(by= 'score',ascending = False)


,filename,score
21,without_test8.JPG,0.667010
14,without_test22.JPG,0.455950
10,without_test19.JPG,0.407936
11,without_test2.JPG,0.370345
3,without_test12.JPG,0.324219
0,without_test1.JPG,0.270005
22,without_test9.JPG,0.250527
12,without_test20.JPG,0.228948
16,without_test3.JPG,0.208358
19,without_test6.JPG,0.203463


In [12]:
model = tf.keras.models.load_model('2020-06-14-1901-Conv2D.model')

In [16]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 117, 117, 128)     2176      
_________________________________________________________________
activation_23 (Activation)   (None, 117, 117, 128)     0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 58, 58, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 58, 58, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 55, 55, 128)       262272    
_________________________________________________________________
activation_24 (Activation)   (None, 55, 55, 128)       0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 27, 27, 128)      